# Pytorch Training With Early Stopping and Progress Bar

## cifar_model.py

In [ ]:
# The content of cifar_model.py

'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm


class FakeReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output


class SequentialWithArgs(torch.nn.Sequential):
    def forward(self, input, *args, **kwargs):
        vs = list(self._modules.values())
        l = len(vs)
        for i in range(l):
            if i == l - 1:
                input = vs[i](input, *args, **kwargs)
            else:
                input = vs[i](input)
        return input


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes))

    def forward(self, x, fake_relu=False):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        if fake_relu:
            return FakeReLU.apply(out)
        return F.relu(out)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x, fake_relu=False):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        if fake_relu:
            return FakeReLU.apply(out)
        return F.relu(out)


class ResNet(nn.Module):
    # feat_scale lets us deal with CelebA, other non-32x32 datasets
    def __init__(self, block, num_blocks, num_classes=10, feat_scale=1, wm=1, dataset='cifar10'):
        super(ResNet, self).__init__()

        assert dataset in ['cifar10', 'celeba128']
        first_stride = {'cifar10': 1, 'celeba128': 2}[dataset]

        widths = [64, 128, 256, 512]
        widths = [int(w * wm) for w in widths]

        self.in_planes = widths[0]
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=3, stride=first_stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.layer1 = self._make_layer(block, widths[0], num_blocks[0], stride=first_stride)
        self.layer2 = self._make_layer(block, widths[1], num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, widths[2], num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, widths[3], num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, widths[3], num_blocks[3], stride=2)
        # self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.pool = nn.AvgPool2d(4)
        self.linear = nn.Linear(feat_scale * widths[3] * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return SequentialWithArgs(*layers)

    def forward(self, x, with_latent=False, fake_relu=False, no_relu=False):
        assert (not no_relu), \
            "no_relu not yet supported for this architecture"
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out, fake_relu=fake_relu)
        out = self.pool(out)
        pre_out = out.view(out.size(0), -1)
        final = self.linear(pre_out)
        if with_latent:
            return final, pre_out
        return final


def ResNet18(**kwargs):
    return ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)


def ResNet18Wide(**kwargs):
    return ResNet(BasicBlock, [2, 2, 2, 2], wd=1.5, **kwargs)


def ResNet18Thin(**kwargs):
    return ResNet(BasicBlock, [2, 2, 2, 2], wd=.75, **kwargs)


def ResNet34(**kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)


def ResNet50(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)


def ResNet101(**kwargs):
    return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)


def ResNet152(**kwargs):
    return ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)


resnet50 = ResNet50
resnet18 = ResNet18
resnet101 = ResNet101
resnet152 = ResNet152

## Training notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision.transforms as transforms
import numpy as np
import torchvision
from cifar_model import resnet50
from tqdm import tqdm
import copy
import torch.nn.functional as F

In [3]:
torch.__version__, torchvision.__version__

('1.4.0', '0.5.0')

In [4]:
device = 'cuda:0'

In [5]:
def train_val(model, optimizer, train_loader, val_loader, epochs, earlystop_patience=-1):
    model.train()
    val_accs = []
    best_val_loss = np.inf
    patience_counter = 0
    for epoch in range(epochs):
        # Train
        train_loss, train_preds, train_labels = [], [], []
        model.train()
        pbar = tqdm(train_loader, position=0, leave=True)
        for i, (data, label) in enumerate(pbar):                                            
            data, label = data.to(device), label.to(device)
            optimizer.zero_grad()
            logits = model(data)
            loss = F.cross_entropy(logits, label)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(logits, dim=1)
            acc = (preds == label).type(torch.FloatTensor).mean()
            train_loss.append(loss.item()), train_preds.append(preds), train_labels.append(label)
            train_loss_mean = torch.Tensor(train_loss).mean()
            train_acc_mean = (torch.cat(train_preds) == 
                              torch.cat(train_labels)).type(torch.FloatTensor).mean().item()
            pbar.set_postfix({'epoch': epoch, 
                              'loss': f'{train_loss_mean:.2f}', 
                              'acc': f'{train_acc_mean:.2f}'})
        # Validation
        model.eval()
        val_loss, val_preds, val_labels = [], [], []
        pbar = tqdm(val_loader, position=0, leave=True)
        for i, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            with torch.no_grad():
                logits = model(data)
                loss = F.cross_entropy(logits, label).item()
                preds = torch.argmax(logits, dim=1)
                acc = (preds == label).type(torch.FloatTensor).mean().item()
                val_loss.append(loss), val_preds.append(preds), val_labels.append(label)
            val_loss_mean = torch.Tensor(val_loss).mean()
            val_acc_mean = (torch.cat(val_preds) == 
                            torch.cat(val_labels)).type(torch.FloatTensor).mean().item()
            pbar.set_postfix({'epoch': epoch, 
                              'val loss': f'{val_loss_mean:.2f}', 
                              'val acc': f'{val_acc_mean:.2f}'})
            
        if val_loss_mean < best_val_loss:
            best_val_loss = val_loss_mean
            patience_counter = 0
            best_model_state = copy.deepcopy(model.state_dict())
        else:
            patience_counter += 1
            if earlystop_patience >= 0 and patience_counter > earlystop_patience:
                break
    if earlystop_patience >= 0:
        model.load_state_dict(best_model_state)

In [11]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
val_loader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
model = resnet50(num_classes=len(trainset.classes))
model = torch.nn.DataParallel(model)
model.to(device);

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=20)

100%|██████████| 100/100 [00:07<00:00, 13.58it/s, epoch=19, val loss=0.49, val acc=0.84]


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=2)

100%|██████████| 100/100 [00:07<00:00, 13.12it/s, epoch=1, val loss=0.25, val acc=0.92]


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=10)

100%|██████████| 100/100 [00:07<00:00, 13.08it/s, epoch=9, val loss=0.25, val acc=0.93]
